In [0]:
# Importar las librerías necesarias
import requests
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
import json

# Crear una sesión de Spark (opcional, si ya estás en un notebook de Databricks, esto se hace automáticamente)
spark = SparkSession.builder.appName("WeatherAPI").getOrCreate()


In [0]:
container = "datos"
account = spark.conf.get("adls.account.name")
datos_path = f"abfss://{container}@{account}.dfs.core.windows.net/datos-api"
def guardar_datos(df, datos_path):
        df = df.repartition(1)
        df.write.format('delta').mode("overwrite").save(f"{datos_path}")   

In [0]:
# URL de la API y tu clave de API (esto es solo un ejemplo, ajusta según la API que utilices)
api_key = "CQPEAKEG8AnpvlveYnsE3s2xGJq9ZQ7T"
language = "es"
details = "true"
metric = "true"
city = "Oviedo"
url = f"http://dataservice.accuweather.com/forecasts/v1/hourly/12hour/{305478}?apikey={api_key}&language={language}&details={details}&metric={metric}"

#url = f"http://dataservice.accuweather.com/forecasts/v1/daily/5day/{locationKey}?apikey={api_key}&"


# Hacer la solicitud a la API
response = requests.get(url)

# Verificar que la respuesta sea exitosa
if response.status_code == 200:
    # Convertir la respuesta a un diccionario de Python
    data = response.json()
else:
    print(f"Error en la solicitud de la API: {response.status_code}")


In [0]:
print(data)

[{'DateTime': '2024-09-18T09:00:00+02:00', 'EpochDateTime': 1726642800, 'WeatherIcon': 12, 'IconPhrase': 'Chubascos', 'HasPrecipitation': True, 'PrecipitationType': 'Rain', 'PrecipitationIntensity': 'Moderate', 'IsDaylight': True, 'Temperature': {'Value': 20.8, 'Unit': 'C', 'UnitType': 17}, 'RealFeelTemperature': {'Value': 18.3, 'Unit': 'C', 'UnitType': 17, 'Phrase': 'Agradable'}, 'RealFeelTemperatureShade': {'Value': 18.3, 'Unit': 'C', 'UnitType': 17, 'Phrase': 'Agradable'}, 'WetBulbTemperature': {'Value': 17.7, 'Unit': 'C', 'UnitType': 17}, 'WetBulbGlobeTemperature': {'Value': 18.7, 'Unit': 'C', 'UnitType': 17}, 'DewPoint': {'Value': 15.5, 'Unit': 'C', 'UnitType': 17}, 'Wind': {'Speed': {'Value': 11.1, 'Unit': 'km/h', 'UnitType': 7}, 'Direction': {'Degrees': 35, 'Localized': 'NE', 'English': 'NE'}}, 'WindGust': {'Speed': {'Value': 16.7, 'Unit': 'km/h', 'UnitType': 7}}, 'RelativeHumidity': 71, 'IndoorRelativeHumidity': 71, 'Visibility': {'Value': 16.1, 'Unit': 'km', 'UnitType': 6}, 'C

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import col, date_format, to_timestamp, expr


# Crear una sesión de Spark
spark = SparkSession.builder.appName("WeatherData").getOrCreate()

# Crear una lista vacía para almacenar los datos formateados
weather_data = []

# Iterar sobre cada elemento en la lista de datos y crear filas (Row) de Spark
for item in data:
    weather_data.append(Row(
        DateTime_prev=item["DateTime"],
        Temperature=item["Temperature"]["Value"],
        RelativeHumidity=item["RelativeHumidity"],
        Rain=item["Rain"]["Value"],
        SolarIrradiance=item["SolarIrradiance"]["Value"]       
    ))

# Crear un DataFrame de Spark a partir de la lista de filas (Row)
df = spark.createDataFrame(weather_data)

df_converted = df.withColumn("DateTime", 
                             date_format(to_timestamp(col("DateTime_prev"), "yyyy-MM-dd'T'HH:mm:ssXXX"), "yyyy-MM-dd HH:mm:ss"))

In [0]:
df_converted.show()


+--------------------+-----------+----------------+----+---------------+-------------------+
|       DateTime_prev|Temperature|RelativeHumidity|Rain|SolarIrradiance|           DateTime|
+--------------------+-----------+----------------+----+---------------+-------------------+
|2024-09-18T09:00:...|       20.8|              71| 1.3|           21.9|2024-09-18 07:00:00|
|2024-09-18T10:00:...|       21.9|              70| 0.0|          601.0|2024-09-18 08:00:00|
|2024-09-18T11:00:...|       22.9|              65| 0.0|          735.0|2024-09-18 09:00:00|
|2024-09-18T12:00:...|       23.6|              63| 0.0|          597.8|2024-09-18 10:00:00|
|2024-09-18T13:00:...|       24.2|              60| 0.0|          845.1|2024-09-18 11:00:00|
|2024-09-18T14:00:...|       25.0|              57| 0.0|          816.7|2024-09-18 12:00:00|
|2024-09-18T15:00:...|       24.4|              59| 0.0|          719.3|2024-09-18 13:00:00|
|2024-09-18T16:00:...|       24.1|              60| 0.4|          439.

In [0]:
# Eliminamos la primera columna y pasamos la ultima a la primera
ultima_columna = df_converted.columns[0]
df_sin_ultima_columna = df_converted.drop(ultima_columna)

columnas = df_sin_ultima_columna.columns
columnas_reordenadas = [columnas[-1]] + columnas[:-1]
df_reordenado = df_sin_ultima_columna.select(columnas_reordenadas)

df_reordenado.show()

+-------------------+-----------+----------------+----+---------------+
|           DateTime|Temperature|RelativeHumidity|Rain|SolarIrradiance|
+-------------------+-----------+----------------+----+---------------+
|2024-09-18 07:00:00|       20.8|              71| 1.3|           21.9|
|2024-09-18 08:00:00|       21.9|              70| 0.0|          601.0|
|2024-09-18 09:00:00|       22.9|              65| 0.0|          735.0|
|2024-09-18 10:00:00|       23.6|              63| 0.0|          597.8|
|2024-09-18 11:00:00|       24.2|              60| 0.0|          845.1|
|2024-09-18 12:00:00|       25.0|              57| 0.0|          816.7|
|2024-09-18 13:00:00|       24.4|              59| 0.0|          719.3|
|2024-09-18 14:00:00|       24.1|              60| 0.4|          439.9|
|2024-09-18 15:00:00|       23.7|              61| 0.0|          389.1|
|2024-09-18 16:00:00|       23.5|              61| 0.0|          220.5|
|2024-09-18 17:00:00|       22.6|              65| 0.0|         

In [0]:
guardar_datos(df_reordenado, datos_path)

In [0]:
%sql
USE CATALOG hive_metastore;
CREATE DATABASE IF NOT EXISTS datos;

CREATE DATABASE IF NOT EXISTS datos_external
MANAGED LOCATION 'abfss://datos@masterabv003sta.dfs.core.windows.net/';

CREATE TABLE IF NOT EXISTS datos_external.api
USING DELTA
LOCATION "abfss://datos@masterabv003sta.dfs.core.windows.net/datos-api";